In [249]:
import json
import pandas as pd
from time import sleep
import asyncio
from solana.rpc.async_api import AsyncClient
from datetime import datetime
from itertools import chain
from pydantic import BaseModel, Json
import requests
from typing import Any, Optional
import re

In [81]:
from dotenv import load_dotenv

In [82]:
load_dotenv("../.env")

True

In [83]:
import os

In [85]:
SSC_RPC_ENDPOINT = os.getenv("SSC_RPC_ENDPOINT")

In [86]:
IDO_TOKEN_ACCOUNT = os.getenv("IDO_TOKEN_ACCOUNT")

In [87]:
USDC_TOKEN_ACCOUNT = os.getenv("USDC_TOKEN_ACCOUNT")

In [88]:
sol_client = AsyncClient(SSC_RPC_ENDPOINT)
ssc_account_data = await sol_client.get_account_info(IDO_TOKEN_ACCOUNT)

### Checking txn level data

In [9]:
ssc_signatures = await sol_client.get_confirmed_signature_for_address2(IDO_TOKEN_ACCOUNT, limit=500)

In [11]:
len(ssc_signatures["result"])

500

In [21]:
ssc_signatures["result"][0]["signature"]

'22TPnGPmsnxY6ePKMykkVA7XHBD5MBgejrQmYhYxpbCrFPokGeRgXQDtNXcgWxxEjw3hKuH59bfqYtS8LCnk8KwR'

In [20]:
ssc_signatures["result"][1]["signature"]

'5T6rtBvsNDepV4UQEmXuLz1jo5B8pJsBEh1B49Y5zTtyTcYB58ALZQ2Zv5vpN42VnLHGMqRfojQmPjDTuYbbDTFo'

In [18]:
sample_txn = await sol_client.get_transaction(ssc_signatures["result"][1]["signature"])

In [25]:
sample_txn["result"]["meta"]["status"]

{'Err': {'InstructionError': [0, {'Custom': 303}]}}

In [154]:
sample_txn_success = await sol_client.get_confirmed_transaction("66v4eUZApDGqwsyY2yP8oNAhdTv4us6dPteportBZMUCyZi9vUx51e8tPLKG6VBoqK3JZNwSr2BpnWCUtigkBWxb")

In [155]:
sample_txn_success["result"]["meta"]["status"]

{'Err': {'InstructionError': [1, {'Custom': 303}]}}

In [156]:
meta_success = sample_txn_success["result"]["meta"]

In [157]:
sample_txn_success["result"]["blockTime"]

1641304801

In [158]:
await sol_client.get_block_time(sample_txn_success["result"]["slot"])

{'jsonrpc': '2.0', 'result': 1641304801, 'id': 10}

In [159]:
datetime.fromtimestamp(1641304799)

datetime.datetime(2022, 1, 4, 19, 29, 59)

In [160]:
meta_success["postTokenBalances"]

[{'accountIndex': 3,
  'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
  'owner': '7qSqQG1P7rG6QdcSv88EG1A5kbb7eFgn97jvtaurhEv3',
  'uiTokenAmount': {'amount': '52029280586420',
   'decimals': 6,
   'uiAmount': 52029280.58642,
   'uiAmountString': '52029280.58642'}},
 {'accountIndex': 4,
  'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
  'owner': '4Tu3a9Vfv3y4qDSGxS58yrmtSJj8kv3fg1y9AcB9SGSY',
  'uiTokenAmount': {'amount': '10002753156',
   'decimals': 6,
   'uiAmount': 10002.753156,
   'uiAmountString': '10002.753156'}}]

In [161]:
meta_success["preTokenBalances"]

[{'accountIndex': 3,
  'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
  'owner': '7qSqQG1P7rG6QdcSv88EG1A5kbb7eFgn97jvtaurhEv3',
  'uiTokenAmount': {'amount': '52029280586420',
   'decimals': 6,
   'uiAmount': 52029280.58642,
   'uiAmountString': '52029280.58642'}},
 {'accountIndex': 4,
  'mint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
  'owner': '4Tu3a9Vfv3y4qDSGxS58yrmtSJj8kv3fg1y9AcB9SGSY',
  'uiTokenAmount': {'amount': '10002753156',
   'decimals': 6,
   'uiAmount': 10002.753156,
   'uiAmountString': '10002.753156'}}]

In [162]:
txn_post_bal = [bal["uiTokenAmount"]["uiAmount"] for bal in meta_success["postTokenBalances"] if bal["mint"] == USDC_TOKEN_ACCOUNT]

In [163]:
txn_pre_bal = [bal["uiTokenAmount"]["uiAmount"] for bal in meta_success["preTokenBalances"] if bal["mint"] == USDC_TOKEN_ACCOUNT]

In [164]:
txn_post_bal

[52029280.58642, 10002.753156]

In [165]:
txn_pre_bal

[52029280.58642, 10002.753156]

In [166]:
[bal["owner"] for bal in meta_success["postTokenBalances"] if bal["mint"] == USDC_TOKEN_ACCOUNT]

['7qSqQG1P7rG6QdcSv88EG1A5kbb7eFgn97jvtaurhEv3',
 '4Tu3a9Vfv3y4qDSGxS58yrmtSJj8kv3fg1y9AcB9SGSY']

In [167]:
[bal["owner"] for bal in meta_success["preTokenBalances"] if bal["mint"] == USDC_TOKEN_ACCOUNT]

['7qSqQG1P7rG6QdcSv88EG1A5kbb7eFgn97jvtaurhEv3',
 '4Tu3a9Vfv3y4qDSGxS58yrmtSJj8kv3fg1y9AcB9SGSY']

In [138]:
[float(0 if pre is None else pre) - float(0 if post is None else post) for pre, post in list(zip(txn_pre_bal, txn_post_bal))]

[-20.0, 20.0]

### Code to fetch all the txns

In [223]:
def fetch_time_diff(ts_start: datetime, ts_end: datetime, time_format: str = "seconds"):
    return round((ts_end - ts_start).total_seconds(), 2)

In [224]:
async def fetch_ido_signatures(token_account: str, batch_count: int, cursor_signature: str=None):
    async with AsyncClient(SSC_RPC_ENDPOINT) as sol_client:
        ssc_signatures = await sol_client.get_confirmed_signature_for_address2(
            token_account, 
            limit=batch_count, 
            before=cursor_signature)
    return ssc_signatures["result"]

In [225]:
async def fetch_all_ido_signature_details(batch_count: int):
    ts_start = datetime.now()
    cursor_signature = None
    full_txn_list = []
    while True:
        txn_list = await fetch_all_ido_txns(
            token_account=IDO_TOKEN_ACCOUNT, 
            batch_count=batch_count, 
            cursor_signature=cursor_signature
        )
        if len(txn_list) == 0:
            print("No txns found...")
            ts_end = datetime.now()
            break
        cursor_signature = txn_list[-1]["signature"]
        print(f"Last txn: {cursor_signature} | Len: {len(txn_list)}")
        full_txn_list.append(txn_list)
    ts_end = datetime.now()
    final_txn_list = list(chain.from_iterable(full_txn_list))
    print(f"{len(final_txn_list)} signatures have been fetched in {fetch_time_diff(ts_start=ts_start, ts_end=ts_end)} secs")
    return final_txn_list

In [79]:
results = await fetch_all_ido_signature_details(batch_count=500)

Last txn: 4KW69AL8rbZ7LmDdZxNuddnEknBAP7RR7waBUKLkmmS8E2j2psXBP4SsM2x7ZyCzp8Ad3p29iMxq3mqdNXtRbNwe | Len: 500
Last txn: 5tRAGxiwFhFema8ZQFhEQpdZAVgWmqGGn131gSQNRBoz2WmgpYZhPW1E5qL9dQhFhWzBxFegw2FyW4q4w8taCZmi | Len: 500
Last txn: 55AsotAPbuMvG667qvtHHMGCspi9Wi2MxgWP51JXg5BtxeVsE9fLMAhs5rYo5sBAWKYTcLnkUVkZS49ee52AKgJx | Len: 500
Last txn: 3VLLqDcwcXeeCkfk5NYfyCzEJrRukP9RApoBFqK9MkciueLJTgSPYNiuW1x3G3sLs6eFokGjwHdKbHRCcfMG4Kon | Len: 500
Last txn: 4Y9F88HYLRvjFsj44oJb7372jyXVpuLUZRciZcYRx2WQqrFXKpomm9Z29Ezqn7DSkQweLhx9Esy1gk78woYw5m5K | Len: 500
Last txn: 3hu8pRAF2QB9RUFkP2UNtAJccXF2XqUUmYmu6LyEkwEav3zhamUFA4tWdacd21QAFWrFMt8YFEHHsFnudoDGGz4C | Len: 500
Last txn: 56fYZWoD8zgVHdL7cpBC4Fx3ShG45jjFJpY8gMgSf168dec13AQcVeg5QAwxdgjWjqABai3DXYMXzrUtNoJ1Wg2h | Len: 500
Last txn: 5tD2CjUbRC1nUUegc22Sz3jkm8qUsTKHR818cZpKqqEznoayfVQLzUsTtisPg6TtedhmaJbFWnbcqQ5DDTgXter1 | Len: 500
Last txn: 2h1Stri876PFmfd89TK7U4AEwJzofjGLdzEmKydezQgzhGNyo2hQsc7V4iweu13hYpi4PXvtVtL4hM4vtbz6nd12 | Len: 500
Last txn: 

In [170]:
results[0]

{'blockTime': 1641306521,
 'confirmationStatus': 'finalized',
 'err': {'InstructionError': [0, {'Custom': 303}]},
 'memo': None,
 'signature': '22TPnGPmsnxY6ePKMykkVA7XHBD5MBgejrQmYhYxpbCrFPokGeRgXQDtNXcgWxxEjw3hKuH59bfqYtS8LCnk8KwR',
 'slot': 114774361}

### Code to parse all the txns

In [89]:
def sanitize_balances(balance: Any):
    return float(0 if balance is None else balance)

In [254]:
def extract_usdc_deposited(pre_balances: list, post_balances: list):
    if len(pre_balances) == 1 or len(post_balances) == 1:
        return 0
    txn_post_bal = [bal["uiTokenAmount"]["uiAmount"] \
                    for bal in post_balances if bal["mint"] == USDC_TOKEN_ACCOUNT]
    txn_pre_bal = [bal["uiTokenAmount"]["uiAmount"] \
                   for bal in pre_balances if bal["mint"] == USDC_TOKEN_ACCOUNT]
    usdc_deposited = [sanitize_balances(balance=pre) - sanitize_balances(balance=post) \
     for pre, post in list(zip(txn_pre_bal, txn_post_bal))][1]
    return usdc_deposited

In [250]:
class IdoTxn(BaseModel):
    usdc_amount: float = 0
    sol_amount: float = 0
    shdw_tokens: float
    depositor: Optional[str]
    time_deposited: datetime
    txn_status: str

In [175]:
def calculate_sol_amount(usdc_amount: float):
    return usdc_amount

In [202]:
status_mapper = {
    "Ok": "success",
    "Err": "failure"
}

In [203]:
# Based on total deposits (52,029,280.58) / 30 Million SHDW tokens
FINAL_SHDW_PRICE = 1.734309352666667

In [252]:
async def fetch_txn_details(txn: str):
    async with AsyncClient(SSC_RPC_ENDPOINT) as sol_client:
        txn_details = await sol_client.get_transaction(txn)
    usdc_deposited = extract_usdc_deposited(
        pre_balances=txn_details["result"]["meta"]["preTokenBalances"],
        post_balances=txn_details["result"]["meta"]["postTokenBalances"]
    )
    time_deposited = datetime.fromtimestamp(txn_details["result"]["blockTime"])
    txn_status = status_mapper[list(txn_details["result"]["meta"]["status"].keys())[0]]
    sol_amount = calculate_sol_amount(usdc_amount=usdc_deposited)
    shdw_tokens = usdc_deposited / final_shdw_price
    depositor = ""
    if usdc_deposited != 0:
        depositor = [bal["owner"] for bal in txn_details["result"]["meta"]["preTokenBalances"] \
                 if bal["mint"] == USDC_TOKEN_ACCOUNT][1]
    ido_txn = IdoTxn(
        usdc_amount=usdc_deposited,
        sol_amount=sol_amount,
        shdw_tokens=shdw_tokens,
        depositor=depositor,
        time_deposited=time_deposited,
        txn_status=txn_status
    )
    return ido_txn.dict()

In [255]:
parsed_txn_list = []
for idx, result in enumerate(results):
#     if idx < 1961:
#         continue
#     print(f"Result signature: {result['signature']}")
    parsed_data = await fetch_txn_details(result["signature"])
    if re.search(r"Shadow", parsed_data["depositor"]):
        print("Skipping SSC withdrawal...")
        continue
    if idx % 100 == 0:
        print(f"Parsed {idx + 1} transactions")
    parsed_txn_list.append(parsed_data)

Parsed 1 transactions
Skipping SSC withdrawal...
Parsed 101 transactions
Parsed 201 transactions
Parsed 301 transactions
Parsed 401 transactions
Parsed 501 transactions
Parsed 601 transactions
Parsed 701 transactions
Parsed 801 transactions
Parsed 901 transactions
Parsed 1001 transactions
Parsed 1101 transactions
Parsed 1201 transactions
Parsed 1301 transactions
Parsed 1401 transactions
Parsed 1501 transactions
Parsed 1601 transactions
Parsed 1701 transactions
Parsed 1801 transactions
Parsed 1901 transactions
Parsed 2001 transactions
Parsed 2101 transactions
Parsed 2201 transactions
Parsed 2301 transactions
Parsed 2401 transactions
Parsed 2501 transactions
Parsed 2601 transactions
Parsed 2701 transactions
Parsed 2801 transactions
Parsed 2901 transactions
Parsed 3001 transactions
Parsed 3101 transactions
Parsed 3201 transactions
Parsed 3301 transactions
Parsed 3401 transactions
Parsed 3501 transactions
Parsed 3601 transactions
Parsed 3701 transactions
Parsed 3801 transactions
Parsed 390

IndexError: list index out of range

In [256]:
len(parsed_txn_list)

19721

In [274]:
parsed_txn_list[-109]

{'usdc_amount': 300.0,
 'sol_amount': 300.0,
 'shdw_tokens': 174.41860465116278,
 'depositor': 'En7xnEFGLQkvz5DKV8Ln6SNbLWbJ7Me2KyQgZpNjztsg',
 'time_deposited': datetime.datetime(2022, 1, 3, 19, 30, 2),
 'txn_status': 'success'}

In [280]:
ssc_ido_txns_df = pd.DataFrame(parsed_txn_list)

In [281]:
ssc_ido_txns_df.head()

,usdc_amount,sol_amount,shdw_tokens,depositor,time_deposited,txn_status
0,0.0,0.0,0.0,,2022-01-04 19:58:41,failure
1,0.0,0.0,0.0,,2022-01-04 19:58:28,failure
2,0.0,0.0,0.0,,2022-01-04 19:58:14,failure
3,0.0,0.0,0.0,,2022-01-04 19:58:01,failure
4,0.0,0.0,0.0,,2022-01-04 19:55:27,failure


In [282]:
ssc_ido_txns_df.query('txn_status == "success"').shape

(18559, 6)

In [283]:
ssc_ido_txns_df.query('txn_status == "success"')

,usdc_amount,sol_amount,shdw_tokens,depositor,time_deposited,txn_status
59,20.000000,20.000000,11.627907,3ZW9VXNriCBKLhSjkA6WMkmYWVfzDGBTHMYn3dB5rtNY,2022-01-04 19:29:59,success
60,313.356184,313.356184,182.183828,GPxK4UCtUp9rPcETBeVtjTje2GCrpA1bGZEJDyqXvkyb,2022-01-04 19:29:59,success
61,374.664532,374.664532,217.828216,4GgGxuSTjiWiyKozK7qJ19C7a6cg6Sk8KLGHy8dFBkV7,2022-01-04 19:29:59,success
62,5000.000000,5000.000000,2906.976744,CjZ1BvauPNUKcRM5hHJQJse9Kc3gTAaCYrXdcWjd5xaX,2022-01-04 19:29:59,success
63,1000.000000,1000.000000,581.395349,6YMs5WVGT7np1yR5Wqgjn3r4UyApTRgQUeCiPJKihHUz,2022-01-04 19:29:59,success
...,...,...,...,...,...,...
19608,86.208556,86.208556,50.121253,3Hh5hhtURB7a2Mq8zJcuLo3oZiSxLDveFffEdoF3bW5Z,2022-01-03 19:30:05,success
19609,48.558871,48.558871,28.231902,CBo6vBPz1A8MK2bSUHvJyX7k2cv5JtGTsNxDp2T4NC3w,2022-01-03 19:30:03,success
19610,50.000000,50.000000,29.069767,2ZFuL7rVYLsfDqdKPU6TRoC6GnQ2ZBLmXNUdd3BxT66P,2022-01-03 19:30:03,success
19611,50.000000,50.000000,29.069767,A49xd7dQBa38oCrnkaX6Hcy1FNDEvT8ZcwnuXET3dWTb,2022-01-03 19:30:03,success


In [331]:
ssc_ido_txns_df["shdw_tokens_recieved"] = ssc_ido_txns_df.usdc_amount / FINAL_SHDW_PRICE

In [335]:
ssc_ido_txns_df.drop(columns=["shdw_tokens"], inplace=True)

### Save data to local disk

In [336]:
ssc_ido_txns_df.to_csv("../data/ssc_ido_txn_data.csv")

### Add few more tables for doing analysis

### Save data to planetscale DB